In [24]:
import pandas as pd
import numpy as np
import torch
import sys

In [25]:
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

sys.path.append('/home/jiaqing/桌面/Fea2Fea/property_process/')
from graph_property import G_property, binning

In [42]:
property_file = pd.read_csv('geometric_graph_800_property.txt', sep = '\t')
edge_idx_file = pd.read_csv('geometric_graph_800_edge_idx.txt', sep = ',',header = None)

x = torch.tensor(np.array(property_file), dtype=torch.float)
edge_idx = torch.tensor(np.array(edge_idx_file), dtype=torch.long)

x_train = x[:,2].reshape((len(x),1))
tmp = np.array(x[:,1])
y = binning(tmp, k = 2,data_len = len(x))
#y = torch.tensor([[1] for i in range(200)])
#print(edge_idx)
data = Data(x=x_train, edge_index=edge_idx.t().contiguous(), y =y)
#print(data.y)
#print(data.x)

In [47]:
num_nodes = len(data.y)
print(num_nodes)
num_train_nodes = int(num_nodes * 0.8)
num_valid_nodes = int(num_nodes * 0.1)
num_test_nodes = int(num_nodes * 0.1)
perm = torch.randperm(num_nodes)
train_idx = perm[:num_train_nodes]
valid_idx = perm[num_train_nodes:(num_train_nodes+num_valid_nodes)]
test_dix = perm[(num_train_nodes+num_valid_nodes):(num_train_nodes+num_valid_nodes+num_test_nodes)]

## debugging
#print(train_idx)
#print(valid_idx)

800


In [48]:
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(1, 256)
        self.conv2 = GCNConv(256,64)
        
        self.linear1 = nn.Linear(64,16)
        self.linear2 = nn.Linear(16, 6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)

        return F.log_softmax(x, dim=1)

In [49]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=5e-4)



def train(data, train_idx):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[train_idx], data.y[train_idx])
    #print(loss)
    loss.backward()
    optimizer.step()
    
def test(data, valid_idx):
    model.eval()
    with torch.no_grad():
        pred = model(data).max(dim=1)[1]
    correct = 0
    correct += pred[valid_idx].eq(data.y[valid_idx]).sum().item()
    valid_acc = correct / len(valid_idx)
    return valid_acc

In [59]:
for epoch in range(400):
    train(data,train_idx)

a = test(data,test_dix)
print(a)

0.7625
